In [9]:
import numpy as np
import pandas as pd

## Load Data

In [10]:
# Define the constants for the scenario
pilot = 'PILOT'
flight = 'FLIGHT'
lead_alt = 15000
wing_alt = 10000
CM_Airspeed = 150

In [11]:
# Load all data sources from a given flight
lead_data = pd.read_csv('Data/Lead/Lead_{}_{}.csv'.format(pilot, flight))
wing_data = pd.read_csv('Data/Wingman/Wing_{}_{}.csv'.format(pilot, flight))
cruise_data = pd.read_csv('Data/CruiseMissiles/CMs_{}_{}.csv'.format(pilot, flight))
workload_data = pd.read_csv('Data/Workload/Workload_{}_{}.csv'.format(pilot, flight))
# comm_data = pd.read_csv('Data/Comm/Comms_{}_{}.csv'.format(pilot, flight))
# surface_data = pd.read_csv('Data/SurfaceThreats/SurfaceThreats_{}_{}.csv'.format(pilot, flight))

## Define the Master DF

In [12]:
Altitude_Col = 'Altitude_Msl'
Airpseed_Col = 'True_Airspeed'
cols_L29s = ['Timestamp', 'SampleDate', 'SampleTime', 'TestCard', 'True_Heading', 'Roll',
        'Latitude', 'Longitude', 'Vertical_Speed', Altitude_Col, Airpseed_Col]
cols_CMs = ['Timestamp', 'SampleDate', 'SampleTime', 'TestCard', 'DisSource',
            'DisTime', 'Aplication', 'EntId', 'Latitude', 'Longitude', 'Heading']

In [13]:
# Make sure Timestamp is sorted and in datetime format
lead_data['Timestamp'] = pd.to_datetime(lead_data['Timestamp'])
wing_data['Timestamp'] = pd.to_datetime(wing_data['Timestamp'])
cruise_data['Timestamp'] = pd.to_datetime(cruise_data['Timestamp'])

lead_data = lead_data.sort_values('Timestamp')
wing_data = wing_data.sort_values('Timestamp')
cruise_data = cruise_data.sort_values('Timestamp')

# Merge wing onto lead (treat lead as "truth")
# This is necessary since the lead aircraft is the primary source of data and the timest stamps may not match perfectly.
sortie_df = pd.merge_asof(
    lead_data[cols_L29s].sort_values('Timestamp'),
    wing_data[cols_L29s].sort_values('Timestamp'),
    on='Timestamp',
    direction='nearest',
    suffixes=('_Lead', '_Wing'),
    tolerance=pd.Timedelta('50ms')  # adjust tolerance as needed
)

# Merge cruise missiles the same way
sortie_df = pd.merge_asof(
    sortie_df,
    cruise_data[cols_CMs].sort_values('Timestamp'),
    on='Timestamp',
    direction='nearest',
    suffixes=('', '_CM'),
    tolerance=pd.Timedelta('50ms')
)

In [14]:
# add Cruise Missile meta data to sortie_df
sortie_df['CM_Altitude_Lead'] = lead_alt
sortie_df['CM_Altitude_Wing'] = wing_alt
sortie_df['CM_Airspeed'] = CM_Airspeed

## Define Intercepts

In [15]:
sortie_df.columns

Index(['Timestamp', 'SampleDate_Lead', 'SampleTime_Lead', 'TestCard_Lead',
       'True_Heading_Lead', 'Roll_Lead', 'Latitude_Lead', 'Longitude_Lead',
       'Vertical_Speed_Lead', 'Altitude_Msl_Lead', 'True_Airspeed_Lead',
       'SampleDate_Wing', 'SampleTime_Wing', 'TestCard_Wing',
       'True_Heading_Wing', 'Roll_Wing', 'Latitude_Wing', 'Longitude_Wing',
       'Vertical_Speed_Wing', 'Altitude_Msl_Wing', 'True_Airspeed_Wing',
       'SampleDate', 'SampleTime', 'TestCard', 'DisSource', 'DisTime',
       'Aplication', 'EntId', 'Latitude', 'Longitude', 'Heading',
       'CM_Altitude_Lead', 'CM_Altitude_Wing', 'CM_Airspeed'],
      dtype='object')

In [ ]:
def is_within_cone(df, role):
    """
    Determine if an aircraft (lead/wingman) is within intercept criteria:
      1) Bank angle within ±10°
      2) Within 1.5 nm aft of CM
      3) Within 30° trailing cone of CM velocity vector
    """

    # --- Column definitions ---
    roll_col = f'Roll_{role}'
    lat_col = f'Latitude_{role}'
    lon_col = f'Longitude_{role}'
    alt_col = f'Altitude_{role}'
    heading_col = f'True_Heading_{role}'

    cm_lat_col = 'Latitude'
    cm_lon_col = 'Longitude'
    cm_alt_col = 'CM_Altitude'
    cm_heading_col = 'Heading'   # you’ll need missile heading in your data

    # --- CONDITION 1: Bank angle ---
    cond1 = df[roll_col].abs() <= 10

    # --- Convert to radians ---
    lat_ac = np.radians(df[lat_col])
    lon_ac = np.radians(df[lon_col])
    lat_cm = np.radians(df[cm_lat_col])
    lon_cm = np.radians(df[cm_lon_col])

    # --- Approx Earth radius in nm ---
    R = 3440.065

    # --- ENU vector from CM → Aircraft (flat Earth approx for short ranges) ---
    dlat = lat_ac - lat_cm
    dlon = lon_ac - lon_cm
    dx = R * np.cos(lat_cm) * dlon     # east displacement [nm]
    dy = R * dlat                      # north displacement [nm]
    dz = (df[alt_col] - df[cm_alt_col]) / 6076.12  # alt diff [nm]
    vec_cm2ac = np.stack([dx, dy, dz], axis=1)

    # --- Missile velocity vector from heading ---
    cm_heading_rad = np.radians(df[cm_heading_col])
    # assume level flight (no vertical velocity)
    vx = np.sin(cm_heading_rad)
    vy = np.cos(cm_heading_rad)
    vz = 0.0
    vec_cm_vel = np.stack([vx, vy, np.full_like(vx, vz)], axis=1)

    # --- CONDITION 2: Aft + distance ---
    dist = np.linalg.norm(vec_cm2ac, axis=1)   # straight-line dist [nm]
    projection = np.sum(vec_cm2ac * vec_cm_vel, axis=1)
    aft_mask = projection < 0                  # behind the CM
    cond2 = (dist <= 1.5) & aft_mask

    # --- CONDITION 3: Inside trailing cone ---
    dot = np.sum(vec_cm2ac * vec_cm_vel, axis=1)
    cos_angle = dot / (dist * np.linalg.norm(vec_cm_vel, axis=1))
    cos_angle = np.clip(cos_angle, -1, 1)      # numerical safety
    angle = np.degrees(np.arccos(cos_angle))
    cond3 = angle <= 30

    # --- CONDITION 4: Heading Sanity Check ---
    heading_diff = np.abs(df[heading_col] - df[cm_heading_col])
    heading_diff = np.where(heading_diff > 180, 360 - heading_diff, heading_diff)
    cond4 = heading_diff <= 20

    intercept_criteria = cond1 & cond2 & cond3 & cond4

    # --- Combine all conditions ---
    return intercept_criteria

Mulltiple CMs ^^, timing / consent, checks for both, order of events .. ?